In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#  Copyright 2024 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

# CT scan acquisition simulation using [gVXR](https://gvirtualxray.sourceforge.io/) and CT reconstruction with [CIL](https://ccpi.ac.uk/cil/)

This example makes use of two open source libraries fro X-ray imaging. The first one is [gVXR](https://gvirtualxray.sourceforge.io/). It is used to simulate realistic radiographic images from a CAD model. The second one is [CIL](https://ccpi.ac.uk/cil/). It implements many CT reconstruction algorithms, including the well-known FDK. The details of the CT scan acquisition are given in the table below. Both FDK and SIRT (iterative method) reconstructions were performed. 

![Reconstructed slice from radiographs simulated at 150 keV](../results/mock_fuel-Zr-recons-FDK-150keV-clamped_mu.png)

| Parameter | Value |
|-----------|-------|
| source-to-object distance (SOD) | 51 m |
| object-to-detector distance (ODD) | 0.5 m |
| source-to-detector distance (SDD) | 51.5 m |
| detector resolution | 2560 &times; 2160 pixels |
| pixel pitch | 7.91 &times; 7.91 &mu;m |
| scintillator | 500 &mu;m of CsI|
| energy response of the detector | ![Plot of the energy response of the detector](../results/mock_fuel-detector-energy_response.png) |
| detector impulse response | ![Plot of the detector impulse response](../results/mock_fuel-detector-LSF.png) |
| sample material composition | ZrO2 &amp; ZrB2 |
| sample material density | 3.23 &amp; 2.43 g/cm<sup>3</sup>|
| number of projection | 1801 |
| first angle | 0&deg; |
| last angle | 180&deg; |
| number of flat images | 50 |

In [ ]:
# Import packages
import os, math
import numpy as np

# Increase the font size in plots
import matplotlib
font = {'weight' : 'bold',
        'size'   : 25}

matplotlib.rc('font', **font)

import matplotlib.pyplot as plt # Plotting

from gvxrPython3 import gvxr # Simulate X-ray images
from gvxrPython3.utils import loadSpectrumSpekpy, visualise
from gvxrPython3 import gvxr2json # Simulate X-ray images

# CT reconstruction using CIL
from cil.io import TIFFStackReader, TIFFWriter
from cil.utilities.display import show2D, show_geometry
from cil.processors import TransmissionAbsorptionConverter
from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.recon import FDK
from cil.optimisation.algorithms import SIRT
from cil.optimisation.functions import IndicatorBox
from cil.plugins.astra.operators import ProjectionOperator
from cil.utilities.jupyter import islicer

In [ ]:
use_Zirconium = True

## Set the simulation parameters

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context")
gvxr.createOpenGLContext();

In [ ]:
# Set up the detector
print("Set up the detector");
gvxr.setDetectorPosition(0.0, 0.5, 0.0, "m");
gvxr.setDetectorUpVector(0, 0, -1);
gvxr.setDetectorNumberOfPixels(2560, 2160);
gvxr.setDetectorPixelSize(7.91, 7.91, "um");

# Set the impulse response of the detector, a convolution kernel
gvxr.setLSF([0.00110698, 0.00122599, 0.00136522, 0.00152954, 0.00172533, 0.00196116, 0.0022487, 0.00260419, 0.00305074, 0.00362216, 0.00436939, 0.00537209, 0.00676012, 0.0087564, 0.01176824, 0.01659933, 0.02499446, 0.04120158, 0.0767488, 0.15911699, 0.24774516, 0.15911699, 0.0767488, 0.04120158, 0.02499446, 0.01659933, 0.01176824, 0.0087564, 0.00676012, 0.00537209, 0.00436939, 0.00362216, 0.00305074, 0.00260419, 0.0022487, 0.00196116, 0.00172533, 0.00152954, 0.00136522, 0.00122599, 0.00110698])

# Set the scintillator
gvxr.setScintillator("CsI", 500, "um");

In [ ]:
# Plot the energy response of the detector
detector_response = np.array(gvxr.getEnergyResponse("keV"))
plt.figure(figsize= (20,10))
plt.title("Energy response of the detector")
plt.plot(detector_response[:,0], detector_response[:,1])
plt.xlabel('Incident energy: E (in keV)')
plt.ylabel('Detector energy response: $\\delta$(E) (in keV)')
plt.tight_layout()
plt.savefig("../results/mock_fuel-detector-energy_response.png", dpi=20)
plt.savefig("../results/mock_fuel-detector-energy_response.pdf", dpi=600)

In [ ]:
# Plot the energy response of the detector
lsf = np.array(gvxr.getLSF())
half_size = len(lsf) // 2
x = np.arange(-half_size, half_size + 1)

plt.figure(figsize= (20,10))
plt.title("One dimensional line spread function (LSF)")
plt.plot(x, lsf)
plt.xlabel('Pixels')
plt.ylabel('Intensity')
plt.tight_layout()
plt.savefig("../results/mock_fuel-detector-LSF.png", dpi=20)
plt.savefig("../results/mock_fuel-detector-LSF.pdf", dpi=600)

In [ ]:
# Create a source
print("Set up the beam")
gvxr.setSourcePosition(0.0,  -51.0, 0.0, "m");
gvxr.usePointSource();
#  For a parallel source, use gvxr.useParallelBeam();

# Set its spectrum, here an almost monochromatic beam, a few percent of harmonics are used
gvxr.addEnergyBinToSpectrum(150, "keV", 16000 * 0.97)
gvxr.addEnergyBinToSpectrum(150 * 2, "keV", 16000 * 0.02)
gvxr.addEnergyBinToSpectrum(150 * 3, "keV", 16000 * 0.01)

# Poisson noise will be enable
gvxr.enablePoissonNoise();

| Property | Matrix | Kernels |
|----------|--------|---------|
| Composition | ZrO2| ZrB2 |
| Shape | Cylinder | Spheres |
| Diameter | 8 to 10 mm  | 0.8 to 1 mm |
| Height  | 10 mm | N/A |
| Theoretical density | 5.68 g/cm<sup>3</sup> | 6.08 g/cm<sup>3</sup> |
| Measured density | 3.23 g/cm<sup>3</sup> | 2.43 g/cm<sup>3</sup> |
| Measured reduction of density | 43% | 60% |

In [ ]:
gvxr.removePolygonMeshesFromSceneGraph()

if not os.path.exists("../results/mock_fuel-matrix.stl") or not os.path.exists("../results/mock_fuel-kernels.stl"): 
    number_of_spheres = gvxr.makeSpheresInCylinder(
            "matrix",         # aCylinderLabel
            50,               # aCylinderNumberOfSectors
            (8.0 + 10.0) / 4, # aCylinderRadius
            5.8,              # aCylinderHeight
            "kernels",        # aSphereLabel
            50,               # aSphereNumberOfRings
            50,               # aSphereNumberOfSectors
            (0.8 + 1.0) / 4,  # aSphereRadius
            0.1,             # aPercentVolume
            "mm");            # aUnitOfLength

    print("Pellet with", number_of_spheres, "kernels")

    gvxr.addPolygonMeshAsInnerSurface("matrix");
    gvxr.addPolygonMeshAsInnerSurface("kernels");

    gvxr.saveSTLfile("matrix", "../results/mock_fuel-matrix.stl");
    gvxr.saveSTLfile("kernels", "../results/mock_fuel-kernels.stl");    
else:
    gvxr.loadMeshFile("matrix", "../results/mock_fuel-matrix.stl", "mm", False)
    gvxr.loadMeshFile("kernels", "../results/mock_fuel-kernels.stl", "mm", False)

    gvxr.addPolygonMeshAsInnerSurface("matrix");
    gvxr.addPolygonMeshAsInnerSurface("kernels");

Matrix_ZrO2_theoretical_density = 5.68
Kernel_ZrB2_theoretical_density = 6.08

Matrix_UO2_theoretical_density = 10.97
Kernel_UB2_theoretical_density = 12.92

if use_Zirconium:
    gvxr.setCompound("matrix", "ZrO2")
    gvxr.setCompound("kernels", "ZrB2")
    gvxr.setDensity("matrix", (1.0 - 43.1338028 / 100) * Matrix_ZrO2_theoretical_density, "g/cm3");
    gvxr.setDensity("kernels", (1.0 - 60.0328947 / 100) * Kernel_ZrB2_theoretical_density, "g/cm3");
else:
    gvxr.setCompound("matrix", "UO2")
    gvxr.setCompound("kernels", "UB2")
    gvxr.setDensity("matrix", (1.0 - 43.1338028 / 100) * Matrix_UO2_theoretical_density, "g/cm3");
    gvxr.setDensity("kernels", (1.0 - 60.0328947 / 100) * Kernel_UB2_theoretical_density, "g/cm3");

print("rho matrix:", gvxr.getDensity("matrix"))
print("rho kernels:", gvxr.getDensity("kernels"))

gvxr.setNodeTransformationMatrix("matrix", [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]);
gvxr.rotateNode("matrix", 90, 1, 0, 0);

gvxr.setNodeTransformationMatrix("kernels", [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]);
gvxr.rotateNode("kernels", 90, 1, 0, 0);

In [ ]:
# Compute an X-ray image
x_ray_image = np.array(gvxr.computeXRayImage()).astype(np.single) / gvxr.getWhiteImage()

In [ ]:
# Display the corresponding X-ray image
plt.imshow(x_ray_image, cmap="gray");
plt.colorbar()
plt.axis('off');

## Simulate the CT acquisition and save the projections

In [ ]:
gvxr.setDetectorNumberOfPixels(2560, 3);

In [ ]:
# for energy in [7, 38, 53, 60, 70, 90, 110, 130, 150]:
for energy in [7, 38, 53, 150]:
    
    gvxr.resetBeamSpectrum();
    gvxr.addEnergyBinToSpectrum(energy, "keV", 16000 * 0.97)
    gvxr.addEnergyBinToSpectrum(energy * 2, "keV", 16000 * 0.02)
    gvxr.addEnergyBinToSpectrum(energy * 3, "keV", 16000 * 0.01)

    mu_matrix = gvxr.getLinearAttenuationCoefficient("matrix", energy, "keV")
    mu_kernels = gvxr.getLinearAttenuationCoefficient("kernels", energy, "keV")
    
    print("mu matrix:", "{0:.2f}".format(mu_matrix), "at", energy, "keV")
    print("mu kernels:", "{0:.2f}".format(mu_kernels), "at", energy, "keV")
    
    transmission = math.exp(-mu_matrix * 0.9 - mu_kernels * 0.09)
    print("transmission at", energy, "keV:", str("{0:.2f}".format(100 * transmission)) + "%")
    
    # For a LaTeX table
    # print(energy, "&" , "{0:.2f}".format(mu_matrix), "&", "{0:.2f}".format(mu_kernels), "&", "{0:.2f}".format(mu_matrix - mu_kernels),"&", str("{0:.2f}".format(100 * transmission)) + "\\%", "\\\\")
    # print("\\hline")
    
    print()
    
    # Poisson noise will be enable
    gvxr.enablePoissonNoise();
    
    if use_Zirconium:
        projection_path = "../results/mock_fuel-Zr-projs-" + str(energy) + "keV"
    else:
        projection_path = "../results/mock_fuel-U-projs-" + str(energy) + "keV"

    # Simulate a CT scan acquisition
    gvxr.computeCTAcquisition(projection_path, # Where to save the projections
                          "screenshots", # Where to save the screenshots
                          1801, # Total number of projections
                          0, # First angle
                          False, # Include the last angle
                          180, # Last angle
                          50, # Number of flat images
                          0, 0, 0, "mm", # Centre of rotation
                          *gvxr.getDetectorUpVector()); # Rotation axis

    # Create the TIFF reader by passing the directory containing the files
    reader = TIFFStackReader(file_name=projection_path, dtype=np.float32)

    # Read in file, and return a numpy array containing the data
    data_original = reader.read()

    # The data is stored as a stack of detector images, we use the CILlabels for the axes
    axis_labels = ['angle','vertical','horizontal']
    
    # Normalisation
    # Not strictly needed as the data was already corrected
    data_normalised = data_original / data_original.max()
    del data_original
    
    # Prevent log of a negative value
    data_normalised[data_normalised<1e-9] = 1e-9

    # Linearisation
    data_absorption = -np.log(data_normalised)    
    del data_normalised
    
    # Create the CIL geoemtry
    geometry = AcquisitionGeometry.create_Cone3D(source_position=gvxr.getSourcePosition("cm"),
                                                 detector_position=gvxr.getDetectorPosition("cm"),
                                                 detector_direction_x=gvxr.getDetectorRightVector(),
                                                 detector_direction_y=gvxr.getDetectorUpVector(),
                                                 rotation_axis_position=gvxr.getCentreOfRotationPositionCT("cm"),
                                                 rotation_axis_direction=gvxr.getRotationAxisCT())

    # Set the angles, remembering to specify the units
    geometry.set_angles(np.array(gvxr.getAngleSetCT()), angle_unit='degree')

    # Set the detector shape and size
    geometry.set_panel(gvxr.getDetectorNumberOfPixels(), gvxr.getDetectorPixelSpacing("cm"))

    # Set the order of the data
    geometry.set_labels(axis_labels)
    
    # Prepare the data for the reconstruction
    acquisition_data = AcquisitionData(data_absorption, deep_copy=False, geometry=geometry)
    acquisition_data.reorder(order='tigre')
    ig = acquisition_data.geometry.get_ImageGeometry()
    
    ig.voxel_size_x = 0.000791
    ig.voxel_size_y = 0.000791
    ig.voxel_size_z = 0.000791

    print(ig)
    # Perform the FDK reconstruction
    fdk =  FDK(acquisition_data, ig)
    recon = fdk.run()

    del data_absorption, acquisition_data
    
    # Save the CT volume as a TIFF stack
    if use_Zirconium:
        TIFFWriter(data=recon, file_name=os.path.join("../results/mock_fuel-Zr-recons-FDK-" + str(energy) + "keV", "out")).write()
    else:
        TIFFWriter(data=recon, file_name=os.path.join("../results/mock_fuel-U-recons-FDK-" + str(energy) + "keV", "out")).write()
        
    recon_as_array = recon.as_array()
    
    font = {'weight' : 'bold',
            'size'   : 12}

    matplotlib.rc('font', **font)

    fig = plt.figure(figsize=(4.6,3.6))
    plt.imshow(recon_as_array[recon_as_array.shape[0] // 2], cmap="gray", extent=[0,(recon_as_array.shape[1]-1)*ig.voxel_size_x,0,(recon_as_array.shape[0]-1)*ig.voxel_size_y])
    
    if energy < 39:
        plt.title(str(energy) + " keV (low energy beam line)")
    else:
        plt.title(str(energy) + " keV (high energy beam line)")

    if use_Zirconium:
        plt.savefig("../results/mock_fuel-Zr-recons-FDK-" + str(energy) + "keV.pdf")
        plt.savefig("../results/mock_fuel-Zr-recons-FDK-" + str(energy) + "keV.png")
    else:
        plt.savefig("../results/mock_fuel-U-recons-FDK-" + str(energy) + "keV.pdf")
        plt.savefig("../results/mock_fuel-U-recons-FDK-" + str(energy) + "keV.png")
    
    fig = plt.figure(figsize=(4.6, 3.6), tight_layout=True)
    if energy == 110:
        plt.imshow(recon_as_array[recon_as_array.shape[0] // 2], cmap="gray", vmin=0.6, vmax=3.0, extent=[0,(recon_as_array.shape[2]-1)*ig.voxel_size_x,0,(recon_as_array.shape[1]-1)*ig.voxel_size_y])
    elif energy == 150:
        plt.imshow(recon_as_array[recon_as_array.shape[0] // 2], cmap="gray", vmin=0.2, vmax=1.8, extent=[0,(recon_as_array.shape[2]-1)*ig.voxel_size_x,0,(recon_as_array.shape[1]-1)*ig.voxel_size_y])
    else:
        plt.imshow(recon_as_array[recon_as_array.shape[0] // 2], cmap="gray", vmin=0, vmax=max(gvxr.getLinearAttenuationCoefficient("matrix", energy, "keV"), gvxr.getLinearAttenuationCoefficient("kernels", energy, "keV")), extent=[0,(recon_as_array.shape[2]-1)*ig.voxel_size_x,0,(recon_as_array.shape[1]-1)*ig.voxel_size_y])

    plt.xlabel("Pixel position (in mm)")
    plt.ylabel("Pixel position (in mm)")
        
        
    plt.colorbar()
    # plt.axis('off');
    
    if energy < 39:
        plt.title(str(energy) + " keV (low energy beam line)")
    else:
        plt.title(str(energy) + " keV (high energy beam line)")
        
    if use_Zirconium:
        plt.savefig("../results/mock_fuel-Zr-recons-FDK-" + str(energy) + "keV-clamped_mu.pdf")
        plt.savefig("../results/mock_fuel-Zr-recons-FDK-" + str(energy) + "keV-clamped_mu.png")
    else:    
        plt.savefig("../results/mock_fuel-U-recons-FDK-" + str(energy) + "keV-clamped_mu.pdf")
        plt.savefig("../results/mock_fuel-U-recons-FDK-" + str(energy) + "keV-clamped_mu.png")
    
    del recon

In [ ]:
# Shutdown the simulation engine
# gvxr.terminate()